In [ ]:

import numpy as np 
import pandas as pd 

In [ ]:
print("Hello Capstone Project Course!")

## WEEK 3 

In [1]:
# Importing the required libraries

from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import folium
import json
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors
import lxml

In [2]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'html.parser')

ConnectionError: HTTPSConnectionPool(host='en.wikipedia.org', port=443): Max retries exceeded with url: /wiki/List_of_postal_codes_of_Canada:_M (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7fd970bb4630>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',))

In [ ]:
table=soup.find('table')
cells=table.find_all('td')


In [ ]:
PostalCode=[]
Borough=[]
Neighbourhood=[]
for i in range(0,len(cells),3):
    PostalCode.append(cells[i].text.strip())
    Borough.append(cells[i+1].text.strip())
    Neighbourhood.append(cells[i+2].text.strip())
    
df=pd.DataFrame(data=[PostalCode,Borough,Neighbourhood]).transpose()
df.columns=["PostalCode","Borough","Neighbourhood"]
df.head()
    
    

In [ ]:
# Remove Not Assigned Borough
df['Borough'].replace("Not assigned", np.nan, inplace=True)
df.dropna(subset=['Borough'],inplace=True)
df.loc[df.Neighbourhood == 'Not assigned', "Neighbourhood"] = df.Borough

In [ ]:
# group the data by postal code and borough and combine the neighbourhood
df_grp=df.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_grp.head()


In [ ]:
df_grp.shape

In [ ]:
df_geo=pd.read_csv('http://cocl.us/Geospatial_data')
df_geo.columns=['PostalCode','Latitude','Longitude']
df_geo.head()

In [ ]:
# join the two tables to get the desired dataframe
df_join = pd.merge(df_grp, df_geo, on=['PostalCode'], how='inner')
df_join.head()

## Explore and cluster the neighborhoods in Toronto


# Use geopy library to get the latitude and longitude values of Toronto, Canada
address='Toronto, Canada'
geolocator=Nominatim(user_agent='Tr_explorer')
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude
print('The geographical coordinates of the city of Toronto are {}, {}.' .format(latitude,longitude))

In [ ]:
## create map of Toronto using latitude and longitude
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat,lng,borough,neighbourhood in zip(df_join['Latitude'],df_join['Longitude'],df_join['Borough'],df_join['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [ ]:
CLIENT_ID = ' ' # your Foursquare ID
CLIENT_SECRET = ' ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
## focus on M1G woburn
df_join.set_index('PostalCode',inplace=True)
neigh_lat=df_join.loc['M1G']['Latitude']
neigh_lng=df_join.loc['M1G']['Longitude']



In [ ]:
LIMIT=100
radius=500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neigh_lat, 
    neigh_lng, 
    radius, 
    LIMIT)
url


In [ ]:
results=requests.get(url).json()

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
venues=results['response']['groups'][0]['items']
#venues

In [ ]:
nearby_venues=json_normalize(venues)
nearby_venues

In [ ]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neigh Lat', 
                  'Neigh Lng', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
SC_data = df_join[df_join['Borough'] == 'Scarborough'].reset_index(drop=True)
SC_data.head()

In [ ]:
SC_venues = getNearbyVenues(names=SC_data['Neighbourhood'],
                                   latitudes=SC_data['Latitude'],
                                   longitudes=SC_data['Longitude']
                                  )

In [ ]:
SC_venues.groupby('Neighborhood').count()

In [ ]:
# one hot encoding
SC_onehot = pd.get_dummies(SC_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
SC_onehot['Neighborhood'] =SC_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [SC_onehot.columns[-1]] + list(SC_onehot.columns[:-1])
SC_onehot = SC_onehot[fixed_columns]

SC_onehot.head()

In [ ]:
SC_grouped = SC_onehot.groupby('Neighborhood').mean().reset_index()
SC_grouped

In [ ]:
num_top_venues = 5

for hood in SC_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = SC_grouped[SC_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = SC_grouped['Neighborhood']

for ind in np.arange(SC_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(SC_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
# set number of clusters
kclusters = 5

SC_grouped_clustering = SC_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(SC_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

SC_merged = SC_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
SC_merged = SC_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

SC_merged # check the last columns!



In [ ]:
SC_merged.dropna(subset=['Cluster Labels'], inplace=True)
SC_merged['Cluster Labels']=SC_merged['Cluster Labels'].astype(int)
SC_merged


In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(SC_merged['Latitude'], SC_merged['Longitude'], SC_merged['Neighbourhood'], SC_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters